In [1]:
from scipy.io import loadmat
import numpy as np
import math
import random

In [2]:
# load indian pines corrected in a array
data=loadmat('Indian_pines_corrected.mat');
data=data['indian_pines_corrected'];
data.shape

#normailize data
for j in range(data.shape[1]):
    maxv=np.max(data[:,j])
    minv=np.min(data[:,j])
    data[:,j]=(data[:,j]-maxv)/(maxv-minv)
data.shape

(145, 145, 200)

In [3]:
# load indian pines gt in an array
data2=loadmat('Indian_pines_gt.mat');
data2=data2['indian_pines_gt'];
data2.shape
# plt.imshow(data2)
# plt.show()

(145, 145)

In [4]:
data2

array([[3, 3, 3, ..., 0, 0, 0],
       [3, 3, 3, ..., 0, 0, 0],
       [3, 3, 3, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [7]:
data =  np.reshape(data,(145*145, 200))
data2 = np.reshape(data2,(145*145,1))

In [8]:
data.shape
data2.shape

(21025, 1)

In [9]:
new_data = np.hstack((data, data2))
new_data.shape

(21025, 201)

In [10]:
# flatans.shape
X_train = np.zeros((1, 201))
X_test = np.zeros((1, 201))
for i in range(16):
    index = np.where(new_data[:,200]==i+1)
    index = np.array(index)
    X_train = np.vstack((X_train, new_data[index[0, 0:int(index.shape[1]/2)], :]))
    X_test = np.vstack((X_test, new_data[index[0, int(index.shape[1]/2):], :]))
    X_train = X_train[1:,:]
    X_test = X_test[1:,:]

In [11]:
X_train.shape

(5106, 201)

In [10]:
X_test.shape

(5113, 201)

In [11]:
from operator import itemgetter

class Softmax:
    def __init__(self, batch_size, epochs, learning_rate, reg_strength):
        self.W = None
        self.batch_size = batch_size
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.reg_strength = reg_strength
        self.weight_update = 'sgd'

    def train(self, X, y):
        n_features = X.shape[1]
        n_classes = y.max() + 1
        self.W = np.random.randn(n_features, n_classes) / np.sqrt(n_features/2)
        config = {'reg_strength': self.reg_strength, 'batch_size': self.batch_size,
                'learning_rate': self.learning_rate }

        graph = np.zeros(self.epochs)
        for epoch in range(self.epochs):
            loss, config = self.sgd(X, y, config)
            graph[epoch] = loss
#             print("Epoch: %s, Loss: %s" % (epoch, loss))
            return graph

    def predict(self, X):
        return np.argmax(X.dot(self.W), 1)

    def loss(self, X, y, W, reg_strength):
        sample_size = X.shape[0]
        predictions = X.dot(W)

        predictions -= predictions.max(axis=1).reshape([-1, 1])
        softmax = math.e**predictions
        softmax /= softmax.sum(axis=1).reshape([-1, 1])
        loss = -np.log(softmax[np.arange(len(softmax)), y]).sum() 
        loss /= sample_size
        loss += 0.5 * reg_strength * (W**2).sum()

        softmax[np.arange(len(softmax)), y] -= 1
        dW = (X.T.dot(softmax) / sample_size) + (reg_strength * W)
        return loss, dW

    # Schotastic gradient descent
    def sgd(self, X, y, config):
        items = itemgetter('learning_rate', 'batch_size', 'reg_strength')(config)
        learning_rate, batch_size, reg_strength = items
         
        random_indices = random.sample(range(X.shape[0]), batch_size)
        X_batch = X[random_indices]
        y_batch = y[random_indices]
        loss, dW = self.loss(X_batch, y_batch, self.W, reg_strength)
    
        self.W -= learning_rate * dW
        return loss, config

In [12]:
X_train_data = X_train[:,0:-1]
Y_train = X_train[:, 200].astype(int)

X_test_data = X_test[:,0:-1]
Y_test = X_test[:, 200].astype(int)

In [13]:
X_train_data.shape

(5106, 200)

In [14]:
reg_strength = 1e-4
batch_size = X_train_data.shape[0]
epochs = 1000
learning_rate = 0.00001
clf = Softmax(batch_size=batch_size, epochs=epochs, learning_rate=learning_rate, reg_strength=reg_strength)
graph = clf.train(X_train_data, Y_train)
y_predict = clf.predict(X_test_data)
# print np.mean(np.equal(y, pred))

In [15]:
correct = np.sum(y_predict == Y_test)
print("%d of %d predictions correct" % (correct, len(y_predict)))

103 of 5113 predictions correct
